# 🧬 AI-GA Meta-Evolution · Colab v3.5

**Launch the complete three-pillar demo** (Clune 2020) in ~2 min — straight from your browser.

| feature | note |
|---------|------|
| Minimal deps | `torch`, `gymnasium`, `openai_agents`, `gradio`, `ray` (GPU optional) |
| LLM mode | set `OPENAI_API_KEY` → online; else Mixtral via Ollama cloud binary |
| Dashboards | Gradio (7862) + FastAPI (8000 → `/docs`) |
| Tests | `pytest -q` ≥ 90 % branch cov < 0.5 s |
| Observability | Prometheus scrape + inline Matplotlib plot |
| Scale-out | Ray remote evaluation; K8s YAML snippet |
| SOC-2 | no secrets on disk, signed-image example |


In [ ]:
#@title ↳ Select runtime { run: "auto", display-mode: "form" }
USE_GPU = False  #@param {type:"boolean"}
import os; os.environ["USE_GPU"] = str(USE_GPU)
print("⚙️  GPU" if USE_GPU else "⚙️  CPU", "runtime selected")

In [ ]:
%%bash --no-stderr
set -Eeuo pipefail
REPO=AGI-Alpha-Agent-v0
[[ -d $REPO ]] || git clone --depth 1 https://github.com/MontrealAI/$REPO.git -q
cd $REPO/alpha_factory_v1/demos/aiga_meta_evolution

# pick torch wheel (CPU vs CUDA)
IDX=$(python - <<PY
import os; print('cu118' if os.getenv('USE_GPU')=='True' else 'cpu')
PY)
python -m pip -q install --upgrade pip
python -m pip -q install torch torchvision --extra-index-url https://download.pytorch.org/whl/$IDX
python -m pip -q install gymnasium[classic_control] gradio==4.* openai_agents ray httpx prometheus-client pytest coverage
echo '✅  Dependencies installed'

In [ ]:
import os, getpass
if not os.getenv("OPENAI_API_KEY"):
    key = getpass.getpass("Paste OPENAI_API_KEY (or press Enter for offline mode): ")
    if key:
        %env OPENAI_API_KEY=$key
        print("🔑  Key set — online mode.")
    else:
        print("🛰️  Offline mode (Mixtral via Ollama fallback).")

In [ ]:
%%bash
cd AGI-Alpha-Agent-v0/alpha_factory_v1/demos/aiga_meta_evolution
pytest -q --disable-warnings --cov=. --cov-branch --cov-report=term-missing:skip-covered || echo '⚠️  Tests < 90 % coverage — investigate'

In [ ]:
import subprocess, threading, re, time, pathlib, sys, os
ROOT = pathlib.Path('AGI-Alpha-Agent-v0/alpha_factory_v1/demos/aiga_meta_evolution').resolve()
os.chdir(ROOT)
proc = subprocess.Popen([sys.executable, 'agent_aiga_entrypoint.py'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                        text=True)
dash_url = None
def _tail():
    global dash_url
    for ln in proc.stdout:
        print(ln, end='')
        if dash_url is None and 'Running on' in ln and 'https' in ln:
            dash_url = re.search(r'https?://[\w./-]+', ln)[0]
            print(f"\n🔗  Dashboard → {dash_url}\n")
threading.Thread(target=_tail, daemon=True).start()
for _ in range(90):
    if dash_url: break
    time.sleep(1)
if not dash_url: print('⏳  Still starting … check logs above.')

## ☎️  FastAPI helper

In [ ]:
import httpx, json, time
API='http://localhost:8000'
print('Health:', httpx.get(API+'/health').json())
httpx.post(API+'/evolve/10')
print('⏩  10 generations scheduled.')

## 📈  Prometheus scrape

In [ ]:
import httpx, re
raw=httpx.get('http://localhost:8000/metrics').text
gen=int(re.search(r'aiga_generations_total (\d+)', raw).group(1))
fit=float(re.search(r'aiga_best_fitness (\d+\.?\d*)', raw).group(1))
print(f'Gen {gen}, best fitness {fit:.2f}')

## 📊  Plot history

In [ ]:
import glob, json, pandas as pd, matplotlib.pyplot as plt, pathlib
ckpts=sorted(glob.glob('checkpoints/evolver_gen*.json'))
if ckpts:
    hist=json.loads(pathlib.Path(ckpts[-1]).read_text())["history"]
    pd.DataFrame(hist, columns=['gen','avg']).plot(x='gen',y='avg',grid=True,figsize=(6,3));
else:
    print('No checkpoints yet.')

---
## 🏗  Kubernetes snippet
```yaml
apiVersion: apps/v1
kind: Deployment
metadata: { name: aiga-demo }
spec:
  replicas: 1
  selector: { matchLabels: { app: aiga-demo } }
  template:
    metadata: { labels: { app: aiga-demo } }
    spec:
      containers:
      - name: orchestrator
        image: ghcr.io/montrealai/alpha-aiga:latest@sha256:<signed>
        envFrom: [{ secretRef: { name: aiga-secrets } }]
        ports:
        - { containerPort: 8000 }
        - { containerPort: 7862 }
```

### 🎯  Next steps — adjust `meta_evolver.py`, extend the curriculum, or hook the REST API into your own autonomous pipelines.